Reference Link: https://blog.keras.io/a-ten-minute-introduction-to-sequence-to-sequence-learning-in-keras.html

Language Translation Model: Conversion from English to French

**Importing of required libraries**

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np


batch_size = 64
epochs = 100
latent_dim = 256 ## feature dimensionality space
num_samples = 10000
data_path = "/content/drive/MyDrive/GeeksforGeeks Sessions/Language Translation Model/fra-eng/fra.txt"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Data Pipeline**

In [ ]:
## Vectorize the data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')


for line in lines[: min(num_samples, len(lines) -1)]:
  input_text, target_text, _ = line.split('\t')
  target_text = '\t' + target_text + '\n'
  
  input_texts.append(input_text)
  target_texts.append(target_text)

  for char in input_text:
    input_characters.add(char)
  
  for char in target_text:
    target_characters.add(char)

In [ ]:
input_texts

['Go.',
 'Go.',
 'Go.',
 'Go.',
 'Hi.',
 'Hi.',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run!',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Run.',
 'Who?',
 'Wow!',
 'Wow!',
 'Wow!',
 'Duck!',
 'Duck!',
 'Duck!',
 'Fire!',
 'Help!',
 'Hide.',
 'Hide.',
 'Jump!',
 'Jump.',
 'Stop!',
 'Stop!',
 'Stop!',
 'Wait!',
 'Wait!',
 'Wait!',
 'Wait.',
 'Wait.',
 'Wait.',
 'Wait.',
 'Begin.',
 'Begin.',
 'Go on.',
 'Go on.',
 'Go on.',
 'Hello!',
 'Hello!',
 'Hello.',
 'Hello.',
 'Hello.',
 'Hello.',
 'I see.',
 'I see.',
 'I try.',
 'I won!',
 'I won!',
 'I won.',
 'Oh no!',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Relax.',
 'Smile.',
 'Smile.',
 'Smile.',
 'Sorry?',
 'Attack!',
 'Attack!',
 'Attack!',
 'Attack!',
 'Buy it.',
 'Buy it.',
 'Buy it.',
 'Buy it.',
 'Cheers!',
 'Cheers!',
 'Cheers!',
 'Cheers!',
 'Eat it.',
 'Eat it.',
 'Get up.',
 'Get up.',
 'Get up.',
 'Go

In [ ]:
target_texts

['\tVa !\n',
 '\tMarche.\n',
 '\tEn route !\n',
 '\tBouge !\n',
 '\tSalut !\n',
 '\tSalut.\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tPrenez vos jambes à vos cous !\n',
 '\tFile !\n',
 '\tFilez !\n',
 '\tCours !\n',
 '\tFuyez !\n',
 '\tFuyons !\n',
 '\tCours\u202f!\n',
 '\tCourez\u202f!\n',
 '\tPrenez vos jambes à vos cous !\n',
 '\tFile !\n',
 '\tFilez !\n',
 '\tCours !\n',
 '\tFuyez !\n',
 '\tFuyons !\n',
 '\tQui ?\n',
 '\tÇa alors\u202f!\n',
 '\tWaouh\xa0!\n',
 '\tWah\xa0!\n',
 '\tÀ terre\xa0!\n',
 '\tBaisse-toi\xa0!\n',
 '\tBaissez-vous\xa0!\n',
 '\tAu feu !\n',
 "\tÀ l'aide\u202f!\n",
 '\tCache-toi.\n',
 '\tCachez-vous.\n',
 '\tSaute.\n',
 '\tSaute.\n',
 '\tÇa suffit\u202f!\n',
 '\tStop\u202f!\n',
 '\tArrête-toi !\n',
 '\tAttends !\n',
 '\tAttendez !\n',
 '\tAttendez.\n',
 '\tAttends !\n',
 '\tAttendez !\n',
 '\tAttends.\n',
 '\tAttendez.\n',
 '\tCommencez.\n',
 '\tCommence.\n',
 '\tPoursuis.\n',
 '\tContinuez.\n',
 '\tPoursuivez.\n',
 '\tBonjour !\n',
 '\tSalut !\n',
 '\t

In [ ]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

In [ ]:
print('Number of samples: ', len(input_texts))
print('Number of unique input tokens: ', num_encoder_tokens)
print('Number of unique output tokens: ', num_decoder_tokens)
print('Max sequence length for the inputs: ', max_encoder_seq_length)
print('Max sequence length for the outputs: ', max_decoder_seq_length)

Number of samples:  10000
Number of unique input tokens:  70
Number of unique output tokens:  93
Max sequence length for the inputs:  14
Max sequence length for the outputs:  59


In [ ]:
input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

In [ ]:
input_token_index

{' ': 0,
 '!': 1,
 '"': 2,
 '$': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '5': 14,
 '7': 15,
 '8': 16,
 '9': 17,
 ':': 18,
 '?': 19,
 'A': 20,
 'B': 21,
 'C': 22,
 'D': 23,
 'E': 24,
 'F': 25,
 'G': 26,
 'H': 27,
 'I': 28,
 'J': 29,
 'K': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'Y': 43,
 'a': 44,
 'b': 45,
 'c': 46,
 'd': 47,
 'e': 48,
 'f': 49,
 'g': 50,
 'h': 51,
 'i': 52,
 'j': 53,
 'k': 54,
 'l': 55,
 'm': 56,
 'n': 57,
 'o': 58,
 'p': 59,
 'q': 60,
 'r': 61,
 's': 62,
 't': 63,
 'u': 64,
 'v': 65,
 'w': 66,
 'x': 67,
 'y': 68,
 'z': 69}

In [ ]:
target_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '%': 4,
 '&': 5,
 "'": 6,
 '(': 7,
 ')': 8,
 ',': 9,
 '-': 10,
 '.': 11,
 '0': 12,
 '1': 13,
 '2': 14,
 '3': 15,
 '5': 16,
 '8': 17,
 '9': 18,
 ':': 19,
 '?': 20,
 'A': 21,
 'B': 22,
 'C': 23,
 'D': 24,
 'E': 25,
 'F': 26,
 'G': 27,
 'H': 28,
 'I': 29,
 'J': 30,
 'K': 31,
 'L': 32,
 'M': 33,
 'N': 34,
 'O': 35,
 'P': 36,
 'Q': 37,
 'R': 38,
 'S': 39,
 'T': 40,
 'U': 41,
 'V': 42,
 'W': 43,
 'Y': 44,
 'a': 45,
 'b': 46,
 'c': 47,
 'd': 48,
 'e': 49,
 'f': 50,
 'g': 51,
 'h': 52,
 'i': 53,
 'j': 54,
 'k': 55,
 'l': 56,
 'm': 57,
 'n': 58,
 'o': 59,
 'p': 60,
 'q': 61,
 'r': 62,
 's': 63,
 't': 64,
 'u': 65,
 'v': 66,
 'w': 67,
 'x': 68,
 'y': 69,
 'z': 70,
 '\xa0': 71,
 '«': 72,
 '»': 73,
 'À': 74,
 'Ç': 75,
 'É': 76,
 'Ê': 77,
 'à': 78,
 'â': 79,
 'ç': 80,
 'è': 81,
 'é': 82,
 'ê': 83,
 'î': 84,
 'ï': 85,
 'ô': 86,
 'ù': 87,
 'û': 88,
 'œ': 89,
 '\u2009': 90,
 '’': 91,
 '\u202f': 92}

In [ ]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), 
    dtype = 'float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype = 'float32')
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype = 'float32')

In [ ]:
type(encoder_input_data)

numpy.ndarray

In [ ]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
  for t, char in enumerate(input_text):
    encoder_input_data[i, t, input_token_index[char]] = 1.
  encoder_input_data[i, t+1:, input_token_index[' ']] = 1.
  for t, char in enumerate(target_text):
    decoder_input_data[i, t, target_token_index[char]] = 1.
    if t > 0:
      decoder_target_data[i, t-1, target_token_index[char]] = 1.
  decoder_input_data[i, t+1:, target_token_index[' ']] = 1.
  decoder_target_data[i, t:, target_token_index[' ']] = 1.


In [ ]:
encoder_input_data[0].shape

(14, 70)

In [ ]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

In [ ]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = batch_size, epochs = epochs, validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 13s 20ms/step - loss: 1.2284 - accuracy: 0.7318 - val_loss: 1.0940 - val_accuracy: 0.7118
Epoch 2/100
125/125 [==============================] - 1s 11ms/step - loss: 0.9429 - accuracy: 0.7478 - val_loss: 0.9944 - val_accuracy: 0.7143
Epoch 3/100
125/125 [==============================] - 1s 11ms/step - loss: 0.8493 - accuracy: 0.7650 - val_loss: 0.8749 - val_accuracy: 0.7482
Epoch 4/100
125/125 [==============================] - 1s 11ms/step - loss: 0.7488 - accuracy: 0.7898 - val_loss: 0.7858 - val_accuracy: 0.7761
Epoch 5/100
125/125 [==============================] - 1s 11ms/step - loss: 0.6722 - accuracy: 0.8067 - val_loss: 0.7291 - val_accuracy: 0.7860
Epoch 6/100
125/125 [==============================] - 1s 12ms/step - loss: 0.6296 - accuracy: 0.8169 - val_loss: 0.6802 - val_accuracy: 0.8061
Epoch 7/100
125/125 [==============================] - 2s 13ms/step - loss: 0.6014 - accuracy: 0.8241 - val_loss: 0.6588 - val_accuracy

In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [ ]:
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [ ]:
for seq_index in range(100):
  input_seq = encoder_input_data[seq_index: seq_index+1]
  decoded_sentence = decode_sequence(input_seq)
  print('-')
  print('Input sequence:', input_texts[seq_index])
  print('Decoded sequence: ', decoded_sentence)

1/1 [==============================] - 0s 36ms/step
-
Input sequence: Go.
Decoded sequence:  C'est pas boniquiss !

1/1 [==============================] - 0s 22ms/step
-
Input sequence: Go.
Decoded sequence:  C'est pas boniquiss !

1/1 [==============================] - 0s 22ms/step
-
Input sequence: Go.
Decoded sequence:  C'est pas boniquiss !

1/1 [==============================] - 0s 23ms/step
-
Input sequence: Go.
Decoded sequence:  C'est pas boniquiss !

1/1 [==============================] - 0s 19ms/step
-
Input sequence: Hi.
Decoded sequence:  Salut.

1/1 [==============================] - 0s 20ms/step
-
Input sequence: Hi.
Decoded sequence:  Salut.

1/1 [==============================] - 0s 21ms/step
-
Input sequence: Run!
Decoded sequence:  File !

1/1 [==============================] - 0s 19ms/step
-
Input sequence: Run!
Decoded sequence:  File !

1/1 [==============================] - 0s 21ms/step
-
Input sequence: Run!
Decoded sequence:  File !

1/1 [=======================